In [21]:
import numpy as np
from ripser import ripser


In [22]:
def coborder_matrix(lv, le):
    """
    Create the coborder matrix for the list of
    vertices lv in the basis given by the list of edges le.
    
    Yes it's quadratic (len(lv) * len(le)) but who cares now.
    
    T : can be compared for equality
    lv: list of T
    le: list of sequence(T)
    """
    columns = len(lv)
    rows = len(le)
    M = np.zeros((rows, columns))
    for c, vertex in enumerate(lv):
        for r, edge in enumerate(le):
            if vertex in edge:
                start, end = edge[0], edge[1]
                M[r,c] = -1 if vertex == start else 1
    return M

import enum
class Operator(enum.Enum):
    BORDER = 0
    CO_BORDER = 1

def from_distance(D, operator=Operator.BORDER, threshold=np.infty):
    """Returns the operator matrix associated to the threshold filtration.
    
    :param D: The distance matrix for your data set.
    :type D: numpy.ndarry, shape = (k,k)
    
    :param operator: which operator matrix to compute.
    :type operator: Operator
    
    :param threshold: edges longer than threshold will not be computed.
    :type threshold: np.single | np.double | float
    
    :return: (matrix associated with (co)-border operator, edges matrix)
    
    :rtype: (numpy.ndarray, numpy.ndarray)
    
    The return matrix is expressed in the lexicographic ordered base.
    For vertices 0,1,...,n the first n+1 columns will be vertices
    and the remaining will be the edges [i,j].
    
    :warning: edges such as [i,i] are excluded
    """
    # edges[r,c] == True if there is an edge from r to c
    # that fits the threshold distance
    under_threshold = (D <= threshold) & (D > 0)
    edges = under_threshold.astype(int)
    # there is an edge between r and c when edges[r,c] == 1
    # but we need to put the correct sign in the (co)border
    # matrix as it is 1 IFF r < c and -1 otherwise
    ones = np.triu(edges)
    oriented_edges = ones - ones.T
    # now we have edges with orientation and we can compute
    # our border matrix
    (vertices_num, _) = D.shape
    basis_dimension = vertices_num * (vertices_num + 1)
    matrix = np.zeros((basis_dimension, basis_dimension), dtype=int)
    iterator = np.nditer(oriented_edges, flags=['multi_index'])
    edges_num = vertices_num *(vertices_num -1)
    while not iterator.finished:
        (start, end) = iterator.multi_index
        #if start == end: # there cannot be an edge on the same vertex
        #    iterator.iternext()
        #    continue
        if iterator[0] == 1:
            # there is an edge from start to end
            column = vertices_num * (start + 1) + end
            matrix[start, column] = 1
            matrix[end, column] = -1
        iterator.iternext()
    # we also have put in the basis edges such as (i,i)
    # which should not be there
    # fortunately this is an easy list of columne/rows to drop
    drop = [vertices_num * (i + 1) + i for i in range(vertices_num)]
    wo_columns = np.delete(matrix, drop, axis=1)
    wo_rows = np.delete(wo_columns, drop, axis=0)        
    return wo_rows if operator is Operator.BORDER else wo_rows.T, oriented_edges

In [23]:
# let's make the distance for a triangle
distance = np.array([
    [0, 3, 4],
    [3, 0, 6],
    [4, 5, 0]
])

co_border, edges = from_distance(distance, operator=Operator.CO_BORDER)
co_border

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 1, -1,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  0, -1,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  1, -1,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [24]:
def circular_coordinates(p):
    """Transform to circular coordinates.
    
    Returns the same coordinates but modulo Z
    and in the range [0,1)
    
    :param p: coordinates
    :type p: numpy.ndarray
    :rtype: numpy.ndarray
    """
    fractional = p - np.trunc(p)
    correction = np.floor(fractional)
    return fractional - correction

In [25]:
from numpy.linalg import pinv as pseudoinverse
# compute the example matrix from the assignment
m = np.array([
    [-1, 1, 0],
    [-1, 0, 1],
    [0, -1, 1]
])
p = pseudoinverse(m)
circular_coordinates(p @ (- np.array([1, 0,0])))

array([3.33333333e-01, 6.66666667e-01, 5.48387791e-17])

In [26]:
def compute(D, alpha, r, prime):
    """
    :param D: Distance matrix
    :param alpha: representative co-cycle
    :param r: threshold
    :param prime: prime field coefficients
    """
    def distance(i, j):
        return D[i,j]
    # alpha is a representative co-cycle and we can do the
    # conversion to our fancy thingy
    representative = Representative(alpha)
    alpha_r = representative.project(distance, threshold)